In [1]:
import pandas as pd
import numpy as np
import cv2
from skimage.io import imread

In [2]:
# 구글 드라이브를 마운트합니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# GroundTruth.csv 파일 읽기
ground_truth = pd.read_csv('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/GroundTruth.csv')

# NewGroundTruth.csv를 생성하기 위한 데이터 저장
new_data = []

In [12]:
# 기존 데이터셋
ground_truth

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# RLE 인코딩 함수
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [5]:
# 각 이미지에 대해 처리
for index, row in ground_truth.iterrows():
    image_name = row['image']
    class_name = None

    # Class 이름 찾기
    for class_type in ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']:
        if row[class_type] == 1:
            class_name = class_type
            break

    # 이미지와 마스크 파일 경로
    image_path = f'/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/images/{image_name}.jpg'
    mask_path = f'/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/masks/{image_name}_segmentation.png'

    # 마스크 이미지 읽기
    mask_image = imread(mask_path, as_gray=True)

    # 마스크 이미지 바이너리화 (이진화)
    mask_image = (mask_image > 0).astype(np.uint8)

    # RLE 인코딩
    encoded_pixels = rle_encode(mask_image)

    # 데이터 저장
    new_data.append([image_name, class_name, encoded_pixels])

In [6]:
# 새로운 DataFrame 생성
new_ground_truth = pd.DataFrame(new_data, columns=['Image', 'Class', 'EncodedPixels'])

# NewGroundTruth.csv 파일로 저장
new_ground_truth.to_csv('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/NewGroundTruth.csv', index=False)

print("NewGroundTruth.csv 파일이 생성되었습니다.")

NewGroundTruth.csv 파일이 생성되었습니다.


In [9]:
# 파일 불러와서 출력
new_ground_truth = pd.read_csv('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/NewGroundTruth.csv')

In [10]:
new_ground_truth

,Image,Class,EncodedPixels
0,ISIC_0024306,NV,4523 31 5117 39 5713 46 6311 49 6907 54 7505 5...
1,ISIC_0024307,NV,20742 18 21339 24 21937 30 22535 35 23134 39 2...
2,ISIC_0024308,NV,39740 7 40339 11 40938 15 41537 18 42137 20 42...
3,ISIC_0024309,NV,69112 9 69709 15 70305 21 70901 27 71499 30 72...
4,ISIC_0024310,MEL,47094 20 47692 24 48290 28 48888 33 49487 36 5...
...,...,...,...
10010,ISIC_0034316,MEL,50578 38 51176 47 51773 54 52371 58 52969 61 5...
10011,ISIC_0034317,MEL,73500 9 74097 19 74695 24 75292 30 75889 39 76...
10012,ISIC_0034318,BKL,54936 8 55527 47 56119 67 56711 80 57307 89 57...
10013,ISIC_0034319,NV,69900 11 70494 19 71090 25 71686 30 72284 33 7...
